# Efficient FastAPI

FastAPI embraces many of the latest Python language features and external libraries, so writing APIs is easier and it's less error prone. Some of those features include:
1. Type hints
2. async / await
3. ASGI servers
4. Pydantic classes

It is recommended that you refresh your knowledge of Python type hinting and Pydantic before continuing with this lesson.

FastAPI will use this type hinting to perform the transformation of the query parameters into the specified data type we put as a hint.

# async / await: Asynchronous IO 

Async IO is a concurrent programming design that allows us to run multiple tasks in an overlapping manner. Do not confuse it with multithreading since async IO is a single-threaded, single-process design. In this case, the term asynchronous refers to the ability to pause while waiting for a trigger while other tasks run in the meantime.

To see this is in action, take a look at the following `synchronous` example

In [10]:
import time

def count(x):
    print(x + 1)
    time.sleep(1)

def main():
    for i in range(3):
        count(i)

s = time.perf_counter()
main()
elapsed = time.perf_counter() - s
print(f"Executed in {elapsed:0.2f} seconds.")

The code above simply prints from 1 to 3 with a one-second pause between numbers. Now, observe the following `asynchronous` example:

In [17]:
%%python
import asyncio
import time

async def count(x):
    print(x)
    await asyncio.sleep(1)

async def main():
    await asyncio.gather(count(1), count(2), count(3))


s = time.perf_counter()
asyncio.run(main())
elapsed = time.perf_counter() - s
print(f"Executed in {elapsed:0.2f} seconds.")

1
2
3
Executed in 1.00 seconds.


Observe the difference between times. In the asynchronous example, there is no pause between 1, 2, and 3. The async count function is asynchronous, which means it has the ability to pause right before the `await` statement inside it. The second thing to observe is that the asynchronous method is much faster because you don't have any dead time until you finish all the concurrent calls.

On the other hand, asyncio.gather creates three concurrent tasks (count(1), count(2), count(3)). They are not going to run simultaneously, but they have the ability to overlap each other.

Roughly, we can see the above example as:
```
        import asyncio
        import time

        async def count(x):
            print(x)
(3, 5, 7)   await asyncio.sleep(1)

        async def main():
(2, 4, 6)   await asyncio.gather(count(1), count(2), count(3))


        s = time.perf_counter()
(1)     asyncio.run(main())                                     
        elapsed = time.perf_counter() - s
        print(f"Executed in {elapsed:0.2f} seconds.")
```

1. Run `main()`
2. Run `count(1)`
3. Print 1, and reach `await asyncio.sleep(1)`. It will tell async: 'Hey, I'll be here waiting, why don't you tell the next concurrent job to start?'
4. Run `count(2)`
5. Print 2, and reach `await asyncio.sleep(1)`. It will tell async: 'Hey, I'll be here waiting, why don't you tell the next concurrent job to start?'
6. Run `count(3)`
7. Print 3, and reach `await asyncio.sleep(1)`. It will see that there are no more concurrent jobs, so it will continue



The same way you are waiting with the `await asyncio.sleep(1)` line, you also have to wait when you scrape data from a website. Most of the time you wait is used for waiting to a single response, and then move to extract the next sample. You can leverage the potential of async and await using the `httpx` library. You can go to the `examples` folder to see two examples: one asynchronous solution and its synchronous counterpart. You can test them to see their differences. 

> ## Asynchronous IO allows a much more efficient solution for processes by running concurrent subprocesses

# ASGI servers

Traditional web frameworks run in a sequential manner, you send the request, the server processes your request, and the response is sent back to you. If we establish a set of rules to the communication between the user and the server, we will (in a nutshell) obtain a __WSGI__ (Web Server Gateway Interface).

Currently, [Gunicorn](https://gunicorn.org/) is the most common used HTTP WSGI for Python, and it works very well with different Python framworks such as Flask or Django, when they want to deploy the app to production.

However, we know that we can also work in an asynchronous manner, send multiple requests at the same time, and then process each of them. If our server works in an asynchronous manner, it becomes an __ASGI__ (Asynchronous Server Gateway Interface). They will leverage the use of `async` and `await` we saw above.

Currently, [Uvicorn](https://www.uvicorn.org/) is one of the main server that supports ASGI. You can take a look at the different frameworks that can work in an ASGI server [here](https://github.com/florimondmanca/awesome-asgi)

<p align=center><img src=images/ASGI.png width=500></p>



# Creating your Second API

For this second API, we are going to accept the name of a celebrity, and we will return the DOB of this celebrity. Let's start setting up the code. Same as in the last notebook, we will create a very simple API and build on top of it. Once again, we will use the magic cell here, but it is not recommended. Make sure to activate the virtual environment where you installed fastapi and uvicorn!


In [4]:
%%python
import fastapi
import uvicorn
api = fastapi.FastAPI() 

@api.get('/')
def index():
    return 'Welcome to the celebrity DOB API!'

if __name__ == '__main__':
    uvicorn.run(api, port=8000, host='127.0.0.1')

# Routing with FastAPI

When you work in a project, your code will be partitioned into smaller scripts, so your program is more organized. In FastAPI we can do the same using Routers. As an example, let's say that you are adding two GET responses to your API, one for a regular DOB request, and another for giving an icon to the tab we will work in:

In [ ]:
%%python
import fastapi
import uvicorn

api = fastapi.FastAPI()

@api.get('/')
def index():
    return 'Welcome to the celebrity DOB API!'

@api.get('/api/dob')
def dob():
    return 'Some random DOB'
    
if __name__ == '__main__':
    uvicorn.run(api, port=8008, host='127.0.0.1')

You can go to `http://127.0.0.1:8008/api/dob` and see the output:
<p align=center><img src=images/Routing_1.png width=300></p>

One way to separate these functions would be having one script for the home page, and another one for additional pages (`/api/dob`)

```
root/
│
├── home.py
├── api
│   └── dob_api.py
└── main.py
```

In [ ]:
# views/home.py

import fastapi

router = fastapi.APIRouter()


@router.get('/')
def index():
    return 'Welcome to the celebrity DOB API!'


In [ ]:
# api/dob_api.py

import fastapi

router = fastapi.APIRouter()

@router.get('/api/dob')
def dob():
    return 'Some random DOB'

In [ ]:
# main.py
import fastapi
import uvicorn
import home

from api import dob_api

api = fastapi.FastAPI()


def configure_routing():
    api.include_router(home.router)
    api.include_router(dob_api.router)


if __name__ == '__main__':
    configure_routing()
    uvicorn.run(api, port=8000, host='127.0.0.1')


Observe that in `home.py` and `api/dob_api.py`, we included `router = fastapi.APIRouter()`, so all files have a common node to look for all requests.

## Try it out

1. Create a directory named views
2. Add a file named home.py in said directory
3. Add the code above
4. Create a directory named api
5. Add a file named dob_api.py in said directory
6. Add the code above to dob_api.py
7. Create a file named main.py ann add the code above
8. Run the code and check everything works
9. Tweak dob_api, so `/api/dob` can accept two query parameters: first name and last name

# Add Pydantic model

Your API will accept request parameters, and Pydantic will be responsible for casting those parameters to the correct format. In this example, we can accept the first name and the last name. 

Pydantic allows you to create a class where each attribute will be casted to the specified data type

In [4]:
from pydantic import BaseModel
from typing import Optional

class Celebrity(BaseModel):
    '''
    This class gives some information about a celebrity. It is intended to be 
    used with the FastAPI example

    Attributes
    ----------
    first_name: str
        The first name of the celebrity
    last_name: str
        The last name of the celebrity
    city: bool
        If True, the API will also returns the city where the celebrity was born
    '''
    first_name: str
    last_name: str
    city: bool = False

uma_thurman = Celebrity(first_name='Uma', last_name='Thurman')    

Let's create another directory to store our models:

```
root/
│
├── home.py
├── api
│   └── dob_api.py
├── models
│   └── celebrities.py
└── main.py
```

Then, we can use this Pydantic model in our FastAPI like shown below:

In [ ]:
# models/celebrities.py

from pydantic import BaseModel
from typing import Optional

class Celebrity(BaseModel):
    '''
    This class gives some information about a celebrity. It is intended to be 
    used with the FastAPI example

    Attributes
    ----------
    first_name: str
        The first name of the celebrity
    last_name: str
        The last name of the celebrity
    city: bool
        If True, the API will also returns the city where the celebrity was born
    '''
    first_name: str
    last_name: str
    city: bool = False

In [ ]:
# api/dob_api.py

import fastapi
from fastapi import Depends

from models.celebrities import Celebrity

router = fastapi.APIRouter()


@router.get('/api/dob/{first_name}')
def dob(celebrity: Celebrity = Depends()):
    full_name = f'{celebrity.first_name} {celebrity.last_name}'
    if celebrity.city:
        report = f'{full_name} was born a random day in a random city'
    else:
        report = f'{full_name} was born a random day'
    return report



One change that you can see is the `Depends` class in the celebrity when used as an argument in `dob`. This will tell the dob function that the variables inside it depend on an external class, so it will know that the attributes of the class are going to be query parameters.

With this, the following URL [http://127.0.0.1:8008/api/dob/Uma?last_name=Thurman](http://127.0.0.1:8008/api/dob/Uma?last_name=Thurman) will return this output:

<p align=center><img src=images/Routing_2.png width=400></p>

Right now, the DOB API is not doing what it's expected to do. Let's create the real function that will scrape the data about the celebrity so that we can return the date through our API. The function that will contain this service will be in the `services` directory:

```
root/
│
├── home.py
├── api
│   └── dob_api.py
├── models
│   └── celebrities.py
├── services
│   └── dob_service.py
└── main.py
```

In [19]:
# services/dob_service.py
import requests
from bs4 import BeautifulSoup
import json
import re

def get_dob(first_name: str, last_name: str, city: bool=False):
    infobox_data = get_infobox(first_name, last_name)
    if not infobox_data:
        return None
    birthday = infobox_data.find('span', {'class': 'bday'})

    report = {'first name': first_name,
              'last_name': last_name,
              'Date of Birth': birthday.text}
    if city:
        birthplace = infobox_data.find('div', {'class': 'birthplace'})
        report['City'] = birthplace.text

    return report

def get_infobox(first_name: str, last_name: str):
    r = requests.get(f'https://en.wikipedia.org/wiki/{first_name}_{last_name}')
    soup = BeautifulSoup(r.text, 'html.parser')
    if soup.find_all('b', text=re.compile('Wikipedia does not have an article with this exact name')):
        return None
    celeb_infobox = soup.find('table', {'class': 'infobox biography vcard'})
    return celeb_infobox.find('td', {'class': 'infobox-data'})

In [28]:
# api/dob_api.py

import fastapi
from fastapi import Depends
import json
from models.celebrities import Celebrity
# from services.dob_service import get_dob
from services.dob_service_async import get_dob
router = fastapi.APIRouter()

@router.get('/api/dob/{first_name}')
def dob(celebrity: Celebrity = Depends()):
    report = get_dob(celebrity.first_name, celebrity.last_name, celebrity.city)
    return fastapi.Response(content=json.dumps(report),
                            media_type='application/json')

If you go to the following URL [http://127.0.0.1:8008/api/dob/Uma?last_name=Thurman&city=True](http://127.0.0.1:8008/api/dob/Uma?last_name=Thurman&city=True), you will get a real API response!
<p align=center><img src=images/Routing_3.png width=500></p>

Also, you can use the requests library to get the json object that has been uploaded to check that everything worked properly:

In [30]:
requests.get('http://127.0.0.1:8008/api/dob/Uma?last_name=Thurman&city=True').json()

{'First name': 'Uma',
 'Last_name': 'Thurman',
 'Date of Birth': '1970-04-29',
 'City': 'Boston, Massachusetts, U.S.'}

# Implementing async and await in your API

As we saw above, FastAPI leverages the use of modern Python tools, especially the asynchronous functions. In order to implement it, we need to simply change the `dob_service.py` file to incorporate these tools. Observe the changes we implemented in the code below, considering that requests doesn't work asynchronously.

In [ ]:
# services/dob_service.py

from bs4 import BeautifulSoup
import json
import re
import httpx
async def get_dob(first_name: str, last_name: str, city: bool=False):
    infobox_data = await get_infobox_async(first_name, last_name)
    if not infobox_data:
        return None
    birthday = infobox_data.find('span', {'class': 'bday'})

    report = {'first name': first_name,
              'last_name': last_name,
              'Date of Birth': birthday.text}
    if city:
        birthplace = infobox_data.find('div', {'class': 'birthplace'})
        report['City'] = birthplace.text

    return report

async def get_infobox_async(first_name: str, last_name: str):
    async with httpx.AsyncClient() as client:
        r = await client.get(f'https://en.wikipedia.org/wiki/{first_name}_{last_name}')
        html = r.text
    soup = BeautifulSoup(html, 'html.parser')
    if soup.find_all('b', text=re.compile('Wikipedia does not have an article with this exact name')):
        return None
    celeb_infobox = soup.find('table', {'class': 'infobox biography vcard'})
    return celeb_infobox.find('td', {'class': 'infobox-data'})

In [ ]:
# api/dob_api.py

import fastapi
from fastapi import Depends
import json
from models.celebrities import Celebrity
from services.dob_service import get_dob


router = fastapi.APIRouter()

@router.get('/api/dob/{first_name}')
async def dob(celebrity: Celebrity = Depends()):
    report = await get_dob(celebrity.first_name, celebrity.last_name, celebrity.city)
    return fastapi.Response(content=json.dumps(report),
                                media_type='application/json')

Notice that, whenever we call an asynchronous function, we need to `await` for it. Similarly, if we need to await because we are calling for an asynchronous function, we need to include it in an `async` function.

The output you see now will be the same as the one obtained previously, however, when you deploy this API, it can provide many more responses much faster. 

# Summary

- You learned how to leverage modern tools of Python in FastAPI
    - Pydantic
    - Async and Await
- You learned the difference between an ASGI and a WSGI server
- You learned how to route API requests to a single node called in main